In [ ]:
import requests
import json
import re
import csv
from bs4 import BeautifulSoup
import time
import jieba
import os
import pandas as pd # 引用套件並縮寫為 pd  

DATA_PATH = os.path.normpath(os.path.join(os.getcwd(), 'PTT_DATA'))
url = 'https://www.ptt.cc/bbs/movie/index{}.html'

filelist = []

In [ ]:
import threading
import queue
page = queue.Queue() #建立queue物件
URL_TEMPLATE = "https://www.ptt.cc/bbs/movie/index{}.html"

In [ ]:
class Do_page(threading.Thread): # 此為python繼承語法

    def __init__(self, name): #constructor 接受 name 參數
        threading.Thread.__init__(self) # initialize super class
        self.name = name # 每條Do_page的名子


    def run(self):# thread啟動後執行函數
        while page.empty() is False:  #檢查titles queue不為空的話，獲取URL後do_page
            current_url = page.get()
            do_page(current_url)

In [ ]:
def get_total_page_cnt():
    URL = "https://www.ptt.cc/bbs/movie/index.html"
    resp = requests.get(URL)

# 這個符號
# ‹-> &lsaquo;
    total_page_cnt = int(re.findall('href="/bbs/movie/index(\d+).html">&lsaquo; 上頁', resp.text)[0]) + 1

    return total_page_cnt 

In [ ]:
def get_list_page(url):
    HOST = "https://www.ptt.cc"
    resp = requests.get(url)
    links = re.findall('<a href="(/bbs/movie/M.+\.html)">.負雷..+</a>', resp.text)
    detail_page_links = [HOST + link for link in links]
    return detail_page_links
def dump_page(url):
    filename = "_".join(url.split('/')[-1].split('.')[:-1]) + '.html'
    filelist.append(filename)
    resp = requests.get(url)
    with open(DATA_PATH + '\\' + filename, 'w', encoding = 'utf8') as f:
        f.write(resp.text)

In [ ]:
def do_page(url):
    for link in get_list_page(url):
        dump_page(link)

In [ ]:
def main():
    for pg in range(get_total_page_cnt(),(get_total_page_cnt()-get_total_page_cnt()),-1):
        url = URL_TEMPLATE.format(pg)
        page.put(url) #加入url，    
    threads = []
    for j in range(4):
        c = Do_page('c' + str(j)) # 建立Do_page物件

        c.start() # 啟動thread

        threads.append(c)

    for thread in threads:
        thread.join() # 主線程必須等到所有threads執行完畢才繼續執行

        
if __name__ == '__main__':
    tStart = time.time() # 起始時間

    print('共爬'+str(get_total_page_cnt())+'頁')
    main()
    tEnd = time.time() # 結束時間

    print('共'+str(len(filelist))+'筆') # 總共抓取標題數

    print("花費",(tEnd - tStart),"秒") # 完成花費時間